# Demo for Credmark Modeling Framework in Jupyter notebook

## 1. Initialize CMF Context

In [1]:
from credmark.cmf.ipython import create_cmf
from credmark.cmf.types import Token, Contract, Address, Account, BlockNumber, Records, JoinType

cmf_param = {
    'chain_id': 1,
    'block_number': 17_000_000,
    'register_utility_global': True
}

context, _model_loader = create_cmf(cmf_param)

## 2. Use Python objects to interact with the blockchain (Web3)

- BlockNumber
- Address
- Contract
- Token

In [2]:
# 2.1 Query timestamp of a block

print(BlockNumber(12312345))
print(BlockNumber(12312345).timestamp_datetime.strftime("%Y-%M-%d %H:%M"))

12312345
2021-07-25 23:07


In [3]:
# 2.2 Get checksum-ed address

usdc_addr = Token("USDC").address
print(usdc_addr, usdc_addr.checksum)
print(Address("0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48"))

0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48 0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48
0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48


In [4]:
# 2.3 Call Contract with auto-loaded ABI

# Fetch the Uniswap V3 pool for USDC and WETH and 0.3% fee
usdc_addr = Address('0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48')
weth_addr = Address('0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2')
uniswap_v3_factory = Contract('0x1F98431c8aD98523631AE4a59f267346ea31F984')
# ABI allows us to know function names and parameters, e.g., getPool(address, address, uint).
uniswap_v3_factory.functions.getPool(
    weth_addr.checksum, usdc_addr.checksum, 3000).call()

'0x8ad599c3A0ff1De082011EFDDc58f1908eb6e6D8'

In [5]:
# 2.4 Fetch ERC20 token info, in this case its symbol & decimals.

crv_token = Token('0xD533a949740bb3306d119CC777fa900bA034cd52')
print(crv_token.symbol, crv_token.decimals)
# Also token-specific values (using ABI).
print(crv_token.functions.rate().call())

CRV 18
6161965695807970181


# 3. Fetch blockchain data (using ETL)

In [6]:
# What tables are available?
print(context.ledger.tables())

['Block', 'Contract', 'Log', 'Receipt', 'Token', 'TokenBalance', 'TokenTransfer', 'Trace', 'Transaction']


In [7]:
# What columns are available?
# We can see that the TokenTransfer table has 9 columns.
context.ledger.table("TokenTransfer").columns

['block_hash',
 'block_number',
 'block_timestamp',
 'from_address',
 'log_index',
 'token_address',
 'to_address',
 'transaction_hash',
 'value']

In [8]:
# 3.1 Simple query to query token transfers (ERC-20 and ERC-721) during a block range
with context.ledger.TokenTransfer as q:
    df = q.select(
        q.columns,
        where=q.BLOCK_NUMBER.between_(context.block_number - 5, context.block_number)).to_dataframe()

display(df)

In [ ]:
# 3.2 A query to aggregate transfer values for ERC-20 and scale them (using Token decimal information)
# sort by number of transfers and total value
with context.ledger.TokenTransfer.as_('tt') as tt:
    with context.ledger.Token.as_('tok') as tok:
        df = tt.select(
            aggregates=[(tok.ADDRESS, 'address'),
                        (tok.SYMBOL, 'symbol'),
                        (tok.DECIMALS, 'decimals'),
                        (tt.field(
                            f'{tt.VALUE.sum_()} / power(10, {tok.DECIMALS})').as_numeric(), 'sum_value'),
                        (tt.VALUE.count_(), 'transfer_count'),
                        ],
            where=tt.BLOCK_NUMBER.between_(
                context.block_number - 5, context.block_number).and_(tok.DECIMALS.is_not_null()),
            group_by=[tok.ADDRESS, tok.DECIMALS, tok.SYMBOL],
            order_by=tt.field('transfer_count').desc().comma_(
                tt.field('sum_value').desc()),
            joins=[(tt.JoinType.INNER, tok, tok.ADDRESS.eq(tt.TOKEN_ADDRESS))]
        ).to_dataframe()

display(df)

,address,symbol,decimals,sum_value,transfer_count
0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,56.6265110300463354,129
1,0xdac17f958d2ee523a2206206994597c13d831ec7,USDT,6,428299.388538000000,59
2,0x0fe0ed7f146cb12e4b9759aff4fa8d34571802ca,PARTY,18,391479.807435718334,10
3,0x4d224452801aced8b2f0aebe155379bb5d594381,APE,18,676.0319955412685330,10
4,0x4f06229a42e344b361d8dc9ca58d73e2597a9f1f,USDT,15,27104.417110000000,7
...,...,...,...,...,...
94,0x34f0915a5f15a66eba86f6a58be1a471fb7836a7,PLSD,12,5.4517266563610000,1
95,0x72377f31e30a405282b522d588aebbea202b4f23,VRN,18,5.0006878242355265,1
96,0x97d4f49eeb0e2c96d5ebaa71ab8418e563ecd9fd,LSD,9,2.4000000000000000,1
97,0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2,MKR,18,0.51323000000000000000,1


## 4. Call Credmark Models

In [ ]:
# 4.1 Fetch Oracle price for a token
crv_token = Token('0xD533a949740bb3306d119CC777fa900bA034cd52')
context.run_model('price.cex', {'base': crv_token})  # type: ignore

{'src': 'chainlink.price-by-registry|CRV / USD|0xb4c4a493AB6356497713A78FFA6c60FB53517c63|v4|True|t:492s|r:0',
 'price': 1.0199944,
 'quoteAddress': '0x0000000000000000000000000000000000000348'}

In [ ]:
# 4.2 Fetch a Uniswap V3 position info, on a specific past block number
context.run_model("uniswap-v3.id", {"id": 355427}, block_number=15931588)

{'id': 355427,
 'lp': '0x297e12154bde98e96d475fc3a554797f7a6139d0',
 'pool': '0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8',
 'tokens': [{'fee': 42.67879997621646,
   'asset': {'address': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'},
   'amount': 0},
  {'fee': 0.03972424166338882,
   'asset': {'address': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2'},
   'amount': 4.353501495802194}],
 'in_range': 'out of range'}

In [ ]:
# 4.3 Fetch a token's info in AAVE pool, with a Token object as input
context.run_model('aave-v2.token-asset', Token('USDC'))

{'token': {'address': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'},
 'aToken': {'address': '0xbcca60bb61934080951369a648fb03df4f96263c'},
 'tokenName': 'USD Coin',
 'supplyRate': 0.019699207938239125,
 'token_price': {'src': 'sushiswap,uniswap-v2,uniswap-v3',
  'price': 0.999503727876781,
  'quoteAddress': '0x0000000000000000000000000000000000000348'},
 'totalDebt_qty': 436478262.013048,
 'stableDebtToken': {'address': '0xe4922afab0bbadd8ab2a88e0c79d884ad337fca6'},
 'totalSupply_qty': 632026965.342317,
 'stableBorrowRate': 0.10534667053615213,
 'totalInterest_qty': 1393.3851720001549,
 'variableDebtToken': {'address': '0x619beb58998ed2278e08620f97007e1116d5d25b'},
 'totalLiquidity_qty': 195548703.329269,
 'variableBorrowRate': 0.03069334107230425,
 'totalStableDebt_qty': 6528799.403668,
 'totalVariableDebt_qty': 429949462.60938,
 'interestRateStrategyContract': {'address': '0x8cae0596bc1ed42dc3f04c4506cfe442b3e74e27'},
 'totalStableDebtPrinciple_qty': 6527406.018496}

In [ ]:
# 4.4 Fetch the history of LP fees for a Uniswap V2 pool
# Reference: https://credmark.com/blog/computing-lp-fees-for-uniswap-v2
# Return data records to be converted as Pandas DataFrame
df = context.run_model(
    'uniswap-v2.lp-fee-history',
    {"pool": "0xB4e16d0168e52d35CaCD2c6185b44281Ec28C9Dc",
        "lp": "0x76E2E2D4d655b83545D4c50D9521F5bc63bC5329"},  # type: ignore
    return_type=Records).to_dataframe()  # type: ignore
df

,transaction_hash,block_number,log_index,from_address,to_address,transaction_value,token0_lp,token1_lp,in_out_amount0,in_out_amount1,token0_lp_current,token1_lp_current,token0_fee,token1_fee
0,0xbfad95a8302a70379e700275aa73da1cbf34c85767d6...,10109485,173,0x0000000000000000000000000000000000000000,0x76E2E2D4d655b83545D4c50D9521F5bc63bC5329,1885295466071170,28028.501581,136.553568,28028.501581,136.553568,0.000000,0.000000,0.000000,0.000000
1,0x542f81e2e779a9b0a102592f12c335cbb95e70410269...,10431987,27,0x0000000000000000000000000000000000000000,0x76E2E2D4d655b83545D4c50D9521F5bc63bC5329,1610636752897116,58890.274007,245.095086,27131.773090,112.919567,30325.359053,126.210934,1433.141863,5.964585
2,0x8b20f67421ea9fededb1613d25fb1487e5b2d61416ad...,10933740,178,0x0000000000000000000000000000000000000000,0x76E2E2D4d655b83545D4c50D9521F5bc63bC5329,75011156151825,83265.224656,235.037703,1749.067435,4.937197,71507.591647,201.848733,10008.565575,28.251773
3,0x5ba608978eb55ae95c4201e9906c69d69a3b4fdcf187...,10933788,276,0x76E2E2D4d655b83545D4c50D9521F5bc63bC5329,0x7FBa4B8Dc5E7616e59622806932DBea72537A56b,-3570943375120111,0.000000,0.000000,-83252.734393,-235.073619,83252.618743,235.073292,0.115650,0.000327
4,0x24bdf717ba0c17f5c1def17ff65ba43ff5dbcf75ad2e...,11275379,244,0x7FBa4B8Dc5E7616e59622806932DBea72537A56b,0x76E2E2D4d655b83545D4c50D9521F5bc63bC5329,3570943375120111,97100.428488,207.994278,97100.428488,207.994278,0.000000,0.000000,0.000000,0.000000
5,0x5e5e5638afa3636f4305581e55e7d4a44e890506a8bc...,13722075,429,0x76E2E2D4d655b83545D4c50D9521F5bc63bC5329,0xB4e16d0168e52d35CaCD2c6185b44281Ec28C9Dc,-607060373770418,369190.438267,77.797603,-75617.318681,-15.934449,309583.910413,65.237026,135223.846536,28.495026
6,0xa260738a336925e4b90597a8cfb315d655609795a4bd...,13747446,284,0x76E2E2D4d655b83545D4c50D9521F5bc63bC5329,0xB4e16d0168e52d35CaCD2c6185b44281Ec28C9Dc,-711331920323926,263078.953999,63.424961,-83077.564421,-20.028935,345160.651586,83.213805,995.866834,0.240091
7,0x40b67eccbc7551f1f4ed42220ee4d8c82fed117bbd47...,14088392,92,0x76E2E2D4d655b83545D4c50D9521F5bc63bC5329,0xB4e16d0168e52d35CaCD2c6185b44281Ec28C9Dc,-180204086482061,193971.600790,77.345132,-16867.095721,-6.725664,204562.366327,81.568143,6276.330184,2.502653
8,0x8b2d0055b00039e8a462987d3008ae337e06dbf71c38...,14093131,175,0x76E2E2D4d655b83545D4c50D9521F5bc63bC5329,0xB4e16d0168e52d35CaCD2c6185b44281Ec28C9Dc,-331575519126992,158963.089075,66.655829,-30278.683633,-12.696348,189153.616151,79.315212,88.156557,0.036965
9,0xaefda9b82e530bfe643791434521002ea1859b337de7...,14124828,258,0x76E2E2D4d655b83545D4c50D9521F5bc63bC5329,0xB4e16d0168e52d35CaCD2c6185b44281Ec28C9Dc,-591862301641682,113050.444737,40.970006,-58238.107895,-21.105761,170989.851544,61.967516,298.701088,0.108251


In [ ]:
# 4.5 Calculate the fair value for a interest rate swap from IPOR protocol
# IPOR is a decentralized interest rate swap protocol.
# It offers derivative contract of fixed rate for floating lending rate from Compound and AAVE.
# This model `ipor.get-swap` calculates the fair price (payFixedRate, or receiveFixedRate)
# for a given IRS of a start timestamp, asset and notional.
# The following example for 0x6B175474E89094C44Da98b954EedeAC495271d0F is DAI
# Reference: https://docs.ipor.io/automated-market-maker/the-automated-market-maker

context.run_model(
    'ipor.get-swap',
    {"timestamp": 1676688179,
        "asset": "0x6B175474E89094C44Da98b954EedeAC495271d0F", "notional": 1000000},
    block_number=16652629)

{'leverage': 10,
 'notional': 1000000,
 'payFixedRate': 0.023870746852871383,
 'payFixedPayoff': 0,
 'receiveFixedRate': 0.02035241630568668,
 'iporIndex_current': 0.022032021901674848,
 'receiveFixedPayoff': 0,
 'iporIndex_inception': 0.022032021901674848,
 'spreadPayFixed_current': 0.001838724951196534,
 'spreadPayFixed_inception': 0.001838724951196534,
 'spreadReceiveFixed_current': -0.001679605595988168,
 'spreadReceiveFixed_inception': -0.001679605595988168}